# TRANSFER LEARNING

In this exercise, We will use the Splits API and its concepts which we looked at in the week 2 lecture videos. Also we will look at some additional ways of loading things using tensorflow hub.





## Import libraries and set up the splits

In [ ]:
# Do this if your tfds is below version 3.x and restart the kernel
!pip install tensorflow-datasets==3.2.1

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from os import getcwd

print(tf.__version__)
print(tfds.__version__)


The next code block will download the mobilenet model from TensorFlow Hub, and use its learned features, extracted as feature_extractor and set to be fine tuned by making them trainable. We have already downloaded it for you but feel free to use the commented part to download the latest version from the tfhub.dev website

In [ ]:
import tensorflow_hub as hub 

model_selection = ("mobilenet_v2", 224, 1280) 
handle_base, pixels, FV_SIZE = model_selection
IMAGE_SIZE = (pixels, pixels)

# You can also use directly to download from the source.

# MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
# feature_extractor = hub.KerasLayer(MODULE_HANDLE,
#                                    input_shape=IMAGE_SIZE + (3,))

# We have already downloaded the data for you
feature_extractor = hub.KerasLayer('mobilenet_feature_vector',input_shape=IMAGE_SIZE + (3,))
feature_extractor.trainable = True  

### Split the dataset
You need to use subsets of the original data, which is entirely in the 'train' split. I.E. 'train' contains 25000 records.

Split it up so that you get

- The first 10% is your 'new' training set
- The last 10% is your validation and test sets, split down the middle 
    - i.e. the first half of the last 10% is validation
    - the second half is test
    
These 3 recordsets should be called `train_examples`, `validation_examples` and `test_examples` respectively

In [ ]:
# EXERCISE: Split the dataset

filePath = f"{getcwd()}/../tmp2"

splits = [#DESCRIBE YOUR SPLITS HERE#]
    
# Remember to use `cats_vs_dogs:4.*.*` as dataset as 4.0 support the new splits api
# https://www.tensorflow.org/datasets/catalog/cats_vs_dogs
# It has been downloaded for you so remember to use the data_dir parameter else it will try to download the dataset and give you error

splits, info = tfds.load(#YOUR CODE HERE)

(train_examples, validation_examples, test_examples) = splits

# This will take some time to print
train_len = len(list(train_examples))
validation_len = len(list(validation_examples))
test_len = len(list(test_examples))
print(train_len)
print(validation_len)
print(test_len)

Expected Output
```
2326
1163
1163
```

In [ ]:
num_examples = 2500
num_classes = 2

In [ ]:
# This will turn the 3 sets into batches
# so we can train
# This code should not be changed

def format_image(features):
    image = features['image']
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, features['label']
    
BATCH_SIZE =  32

train_batches = train_examples.shuffle(num_examples).map(format_image).batch(BATCH_SIZE)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE)

In [ ]:
# The new model will take the features from the mobilenet via transfer learning
# And add a new dense layer at the bottom, with 2 classes -- for cats and dogs

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(2, activation='softmax')
])



In [ ]:
# EXERCISE: Train the model

# Compile the model with adam optimizer and sparse categorical crossentropy, 
# and track the accuracy metric
    
model.compile(#YOUR CODE HERE)

# Train it for a number of epochs. You should not need many (max 5)
# Train on the train_Batches, and validation on the validation_batches
    
EPOCHS = #YOUR CODE HERE

history = model.fit(#YOUR CODE HERE#)
    
model.summary()

In [ ]:
# EXERCISE: Evaluate the model

# Evaluate the model on the test batches
eval_results = model.evaluate(#YOUR CODE HERE#)

# And print the results. You should have >90% accuracy
for metric, value in zip(model.metrics_names, eval_results):
    print(metric + ': {:.4}'.format(value))

# Submission Instructions

In [ ]:
# Now click the 'Submit Assignment' button above.

# When you're done or would like to take a break, please run the two cells below to save your work and close the Notebook. This frees up resources for your fellow learners.

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
<!-- Shutdown and close the notebook -->
window.onbeforeunload = null
window.close();
IPython.notebook.session.delete();